In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# model_name = "huangtuoyue/GPT2-GOT2-finetuned"
model_name = "huangtuoyue/GPT2-large-GOTfinetuned_v3"

generator = pipeline("text-generation", model=model_name, tokenizer=model_name,\
    do_sample=True, max_length=100, temperature=0.9, top_p = 0.9, pad_token_id = 50256, repetition_penalty=1)

output = generator("Once upon a time")

In [2]:
output = generator("Winter is coming. ")
print(output)

[{'generated_text': 'Winter is coming.   A cold wind blew off the trees, and the wind picked up a few drops on Jon\'s boots. He turned and ran, racing back toward the barracks. The wind had stopped, and darkness had spread across the yard. " You\'re my prisoner, " Ser Alliser Thorne snarled. " Tell them that, boy. " Jon froze. " Aye, they called me ‘prisoner,\' " he said quickly, "but'}]


In [22]:
from nltk.tokenize.treebank import TreebankWordDetokenizer

In [37]:
TreebankWordDetokenizer().detokenize(["hello", "test"])

'hello test'

In [48]:
f = open("./context_v3.txt","w")
incorrect_generation = 0
total = 0
context = "winter is coming. Jon said "
print(context)
f.write(context)
ITERATION = 3
BOS = "[BOS]"
EOS = "[EOS]"
i = 0
options = []
pred_dialogue = False
while i < ITERATION:
    prev_context_end = len(context)
    context_list = context.split()
    if len(context_list) < 50: # used last 50 words to generate new context
        run_context = TreebankWordDetokenizer().detokenize(context_list)
    else: 
        run_context = TreebankWordDetokenizer().detokenize(context_list[-50:])
    if pred_dialogue:
        run_context += " [BOS]"
    pred = generator(run_context)[0]["generated_text"]
    # print(pred)
    total += 1
    # check [BOS]
    words = pred.split()
    eos_list = []
    bos_list = []
    for j, word in enumerate(words): 
        if word == BOS: 
            bos_list.append(j)
        elif word == EOS:
            eos_list.append(j)
    if len(eos_list) == len(bos_list) and len(eos_list) == 0 and not pred_dialogue:
        # a new context 
        context = pred
        print(context)
        f.write(context)
        print("context continue\n")
    elif len(eos_list) != len(bos_list): 
        print("invalid \n")
        #discard
        incorrect_generation += 1
    else:
        # valid bos eos position
        valid = True 
        for j, value in enumerate(bos_list): 
            if eos_list[j] < value: 
                valid = False
            if j > 0 and value < eos_list[j-1]: 
                # bos bos eos
                valid = False
        if not valid: 
            print("invalid\n")
            incorrect_generation += 1
        else:
            # valid option
            option = TreebankWordDetokenizer().detokenize(words[bos_list[0]+1:eos_list[0]])
            options.append(option)
            pred_dialogue = True
            if len(options) == 3: 
                print("============= select dialogue ============")
                f.write("============= select dialogue ============")
                print("A:{}; \nB:{}; \nC: {}".format(options[0], options[1], options[2]))
                d = {"A": options[0], "B": options[1], "C": options[2]}
                f.write(str(d))
                valid = False
                while not valid:
                    user_input = input()
                    if user_input in ["A", "B", "C"]: 
                        valid = True 
                
                print(d[user_input])
                f.write(d[user_input])
                context += (d[user_input])
                options = []
                i += 1
                pred_dialogue = False

f.close()

winter is coming. Jon said 
invalid 

invalid 

invalid 

invalid 

invalid 

============= select dialogue ============
A:The others are not so much. The Wall has men on it whose mothers were whores.; 
B:My lord, Lady Stark is waiting for me. She ’ s not going to let me leave Winterfell, not for a day or two. So I ’ ll stay a day and a night with him. Have you seen the Wall?; 
C: Yes, my lord. The Stark wordsmiths have done a man a kindness. They've carved his father ’ s brow into the likeness of a direwolf,
Yes, my lord. The Stark wordsmiths have done a man a kindness. They've carved his father ’ s brow into the likeness of a direwolf,
winter is coming. Jon said Yes, my lord. The Stark wordsmiths have done a man a kindness. They've carved his father ’ s brow into the likeness of a direwolf, he said. It ’ s the same as the one Jon saw on the kingsroad, only bigger.The next morning Jon heard Samwell Tarly declare that the middle name was bellied. He had ridden past the end of the world

In [49]:
print(incorrect_generation, total)

15 39


# New Method

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, StoppingCriteriaList, StoppingCriteria
# from nltk.tokenize import word_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer

tokenizer = AutoTokenizer.from_pretrained("huangtuoyue/GPT2-large-GOTfinetuned_v5")
model = AutoModelForCausalLM.from_pretrained("huangtuoyue/GPT2-large-GOTfinetuned_v5")

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

In [ ]:
# for stop dialogue generation
class KeywordsStoppingCriteria(StoppingCriteria):
    def __init__(self, keywords_ids:list):
        self.keywords = keywords_ids

    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        if input_ids[0][-1] in self.keywords:
            return True
        return False

# for context generation #
# 1. stop if we get [BOS] -- Jon talking
stop_context_words = ['[BOS]']
stop_context_ids = [tokenizer.encode(w)[0] for w in stop_context_words]
stop_context_criteria = KeywordsStoppingCriteria(stop_context_ids)
# 2. exclude EOS
bad_context_words = ['[EOS]']
bad_context_ids = tokenizer(bad_context_words, add_special_tokens=False).input_ids

# for dialogue generation #
# 1. stop if we get [EOS] -- Jon stops talking
stop_dialogue_words = ['[EOS]']
stop_dialogue_ids = [tokenizer.encode(w)[0] for w in stop_dialogue_words]
stop_dialogue_criteria = KeywordsStoppingCriteria(stop_dialogue_ids)
# 2. exclude BOS
bad_dialogue_words = ['[BOS]', ' "', ' Jon', ' said', ' he', ' Snow']
bad_dialogue_ids = tokenizer(bad_dialogue_words, add_special_tokens=False).input_ids

def generate(text, pred_dialogue):

    encoded_input = tokenizer(text, return_tensors='pt').input_ids
    
    # for context generation
    if not pred_dialogue: 

      outputs = model.generate(encoded_input.to(device), do_sample=True, min_length=60, max_new_tokens=150, pad_token_id = 50256,
                              temperature=0.95, top_p = 1, repetition_penalty = 1.1,
                              stopping_criteria=StoppingCriteriaList([stop_context_criteria]), bad_words_ids = bad_context_ids)
    
    # for dialogue generation
    else:
      outputs = model.generate(encoded_input.to(device), do_sample=True, min_length=3, max_new_tokens=50, pad_token_id = 50256, 
                              temperature=1.1, top_p = 1, repetition_penalty = 1, 
                              stopping_criteria=StoppingCriteriaList([stop_dialogue_criteria]), bad_words_ids = bad_dialogue_ids)
    
    # deceode outputs and keep speical_tokens
    res = tokenizer.batch_decode(outputs, skip_speical_tokens=False)
    return res[0]

In [ ]:
# Introductions #
# text = "Jon grinned and reached under the table to ruffle the shaggy white fur. Inside, one of the guards looked at him and said"
# text = 'Ygritte:"You know nothing, Jon Snow." Jon said'
# text = "Ben clapped Jon on the shoulder."
# text = 'Daenerys: "Hello, Jon Snow. It is a pleasure to meet you." Jon'
text  = "Jon Snow had been travelling for weeks, speaking to people in the villages and towns he visited.[BOS]"
# res, outputs = generate(text, True)
# print(res)

In [ ]:
f = open("./play.txt","w")
incorrect_generation = 0
total = 0
context = "Jon Snow had been travelling for weeks, speaking to people in the villages and towns he visited."
print(context)
f.write(context)
ITERATION = 5
BOS = "[BOS]"
EOS = "[EOS]"
i = 0
options = []
pred_dialogue = False

while i < ITERATION:
    prev_context_end_idx = len(context)
    context_list = context.split()

    # check context length #
    if len(context_list) < 50: # used last 50 words to generate new context
        run_context = TreebankWordDetokenizer().detokenize(context_list)
        last_idx = len(run_context)

    else: 
        run_context = TreebankWordDetokenizer().detokenize(context_list[-50:])
        last_idx = len(run_context)
    
    # dialogue generation #
    if pred_dialogue:
        # print(run_context)
        pred = generate(run_context, pred_dialogue)
    # context generation #
    else:
        # print(run_context)
        pred = generate(run_context, pred_dialogue)
    
    # count total generation time #
    total += 1

    # check if generation includes [BOS] & [EOS] #
    words = pred.split()
    eos_list = []
    bos_list = []
    for j, word in enumerate(words): 
        if word == BOS: 
            bos_list.append(j)
        elif word == EOS:
            eos_list.append(j)
    
    # Validity for context generation #
    if words[-1] == '[BOS]' and not pred_dialogue: # correct context
    # if '[BOS]' in words: # correct context
        context = pred
        # print(context)
        print(context[last_idx:-5] + ":")
        print("\n")
        f.write(context[last_idx:-5]+ ":")
        f.write("\n")

        pred_dialogue = True

    elif words[-1] != '[BOS]' and not pred_dialogue: # no [BOS]
        # print("Invalid Context: no BOS or too long \n")
        incorrect_generation += 1

        pred_dialogue = False

    # Validity for dialogue generation #
    elif pred_dialogue:
        if words[-1] != '[EOS]':
            # print("Invalid Dialogue: no EOS or too long \n")
            incorrect_generation += 1

        elif words[-1] == '[EOS]': # correct dialogue
            option = TreebankWordDetokenizer().detokenize(words[bos_list[-1]+1:eos_list[-1]])
            options.append(option)
            pred_dialogue = True
            if len(options) == 3: 
                print("\n")
                print("============= Select Dialogue ============")
                print("A:{} \nB:{} \nC: {}".format(options[0], options[1], options[2]))
                d = {"A": options[0], "B": options[1], "C": options[2]}
                valid = False

                user_input = input()
                # while not valid:
                #     user_input = input()
                #     if user_input in ["A", "B", "C"]: 
                #         valid = True
                print("Choice Selected: " + user_input)
                print("============= Context Contined ============")
                print("\n")

                f.write("\n")
                f.write("============= Select Dialogue ============")
                f.write("\n")
                f.write("A:{} \nB:{} \nC:{}".format(options[0], options[1], options[2]))
                f.write("\n\n")
                f.write("Choice Selected: " + user_input + "\n")
                f.write("============= Context Contined ============")
                f.write("\n\n")

                context += (" " + d[user_input] + " [EOS]")
                # print(context)
                options = []
                i += 1
                pred_dialogue = False


f.close()

In [ ]:
print(incorrect_generation, total)